In [ ]:
#| default_exp chatbot_tab

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox, Stack, Layout
from traitlets import HasTraits
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
#from jupyter_mentor.chatbot import ChatBot, ChatBotModel, ChatBotView
from jupyter_mentor.madlib import MadLibModel, MadLibView
from jupyter_mentor.llm import FileModel

In [ ]:
#|export
class ChatBotModel(FileModel):

    selected_index = traitlets.Int()
    bot_identity = traitlets.Unicode()
    human_identity = traitlets.Unicode()
    bot_template = traitlets.Unicode()

    def __init__(self, madlibs_list):
        super().__init__()

        self.names = []
        self.madlib_models = []
        for value in madlibs_list:
            model = MadLibModel(**value)
            self.names.append(model.name)
            traitlets.link((self, 'bot_identity'), (model, 'bot_identity'))
            traitlets.link((self, 'human_identity'), (model, 'human_identity'))
            traitlets.link((self, 'bot_template'), (model, 'bot_template'))
            self.madlib_models.append(model)

        self.human_template = "{input_text}"
        self.human_identity = "I am a student"
        self.bot_template = ""
        self.bot_identity = "Your are a tutor."
        
    def prompt(self):
        chat_prompt = ChatPromptTemplate.from_messages(
            [self.madlib_models[self.selected_index].bot_message_prompt, self.madlib_models[self.selected_index].human_message_prompt]
        )
        keys = self.madlib_models[self.selected_index].variables
        values = self.madlib_models[self.selected_index].values
        kwargs = {key:value for key, value in zip(keys,values)}
        prompt = chat_prompt.format_prompt( **kwargs )
        bot = self.llm.invoke(prompt).content
        interm= [desc + ' ' + val for desc, val in zip(self.madlib_models[self.selected_index].descriptions, self.madlib_models[self.selected_index].values)]
        user = "\n".join(interm)
        return (user, bot)
        
    def prompt_with_kwargs(self, kwargs):
        ret = self.llm.invoke(self.chat_prompt.format_prompt(**kwargs))
        return ret.content

In [ ]:
madlibs_list = [
    { 'name': 'Open Prompt',
     'template': '{input_text}',
        'variables': ['input_text'],
        'descriptions': [''],
        'placeholders': ['Message AI chatbot...']
    }, {'name': 'Metaphor',
        'template': "I'm having trouble understanding {concept}. Please explain it as a metaphor",
        'variables': ['concept'],
        'descriptions': ["I'm having trouble understanding"],
        'placeholders': ['']
}]

In [ ]:
chatbot_model = ChatBotModel(madlibs_list)

In [ ]:
chatbot_model.selected_index = 1

In [ ]:
chatbot_model.madlib_models[1].values = ['evolution']

In [ ]:
chatbot_model.prompt()

["I'm having trouble understanding"]
['evolution']
["I'm having trouble understanding evolution"]


("I'm having trouble understanding evolution",
 "Sure! Imagine a tree that starts as a single seed. Over time, this tree grows branches and leaves, each one slightly different from the others due to variations in sunlight, water, and nutrients. Some branches grow stronger and more successful, while others may wither and die.\n\nIn the same way, evolution is like a tree of life, where all living organisms are connected through a process of gradual change and adaptation. Just as the tree's branches represent the diversity of life, different species evolve and change over time, with some adaptations proving to be more beneficial for survival and reproduction.\n\nThese beneficial traits are passed down to future generations, leading to the development of new species that are better suited to their environment. This process of natural selection is like the tree pruning its branches, allowing the strongest and most successful ones to thrive and grow.\n\nIn essence, evolution is the ongoing j

In [ ]:
chatbot_model.madlib_models[0].variables

['input_text']

In [ ]:
#|export
class ChatBotView(widgets.VBox):
    """ This view also exists in 04_chatbot, but wea
    are overwriting it here
    """
    
    def __init__(self):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat = Textarea(
            disabled = True,
            layout=Layout(width='90%', height='400px')
        )
        self.radio_buttons = widgets.RadioButtons()
        self.hbox = widgets.HBox()
        self.default = widgets.Text(
            placeholder='Message AI chatbot...',
            #layout=Layout(width='100%')
        )
        self.stack = widgets.Stack()        
        self.submit_button = widgets.Button(
            value=False,
            disabled=False,
            button_style='success',
            icon='arrow-circle-right' 
        )
        self.hbox.children = (self.stack, self.submit_button)
        self.children = (self.chat, self.radio_buttons, self.hbox ) 

In [ ]:
chatbotview = ChatBotView()
chatbotview

ChatBotView(children=(Textarea(value='', disabled=True, layout=Layout(height='400px', width='90%')), RadioButt…

In [ ]:
#|export
class ChatBot(ChatBotView):
    """ This view also exists in 04_chatbot, but wea
    are overwriting it here
    """
    selected_index = traitlets.Int()
    
    def __init__(self, model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()
        self.model = model
       
        self.radio_buttons.options = [(name, i) for i, name in enumerate(self.model.names)]
        traitlets.link((self, 'selected_index'), (self.radio_buttons, 'value'))

        madlib_views = []
        for model in self.model.madlib_models:
            view = MadLibView(model)
            madlib_views.append(view)
            self.stack.children += (view, )

        traitlets.link((self, 'selected_index'), (self.stack, 'selected_index'))
        traitlets.link((self, 'selected_index'), (self.model, 'selected_index'))

        self.submit_button.on_click(self.on_click)

    def on_click(self, change):
        user, bot = self.model.prompt()
        self.chat.value = "USER: " + user + '\n\n' + "CHATBOT: " + bot + '\n\n'
        self.selected_index = 0
        for child in self.stack.children[self.selected_index].children:
            child.value = ''


In [ ]:
chatbot = ChatBot(chatbot_model)
chatbot

ChatBot(children=(Textarea(value='', disabled=True, layout=Layout(height='400px', width='90%')), RadioButtons(…

In [ ]:
chatbot.model.selected_index

0

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

/opt/homebrew/Caskroom/miniconda/base/envs/jupyter-mentor/lib/python3.11/site-packages/nbdev/export.py:54: UserWarning: Notebook '/Users/nabrewer/Repos/Jupyter4Science/jupyter-mentor/nbs/00_educator.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
/opt/homebrew/Caskroom/miniconda/base/envs/jupyter-mentor/lib/python3.11/site-packages/nbdev/export.py:54: UserWarning: Notebook '/Users/nabrewer/Repos/Jupyter4Science/jupyter-mentor/nbs/00_main.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
/opt/homebrew/Caskroom/miniconda/base/envs/jupyter